# データが保存された複数のcsvファイルを読み込み、必要な計算をして新しいcsvファイルを生成する。

  - ***csvファイルは年毎***に生成されており、不要なフィールドも含んでいる。
  - csvファイルは***Regionというフィールドを含んでおり、4種類(1,2,3,4)の値***が各レコードに設定されている。
  - レコードは4種類より多いため、***Regionというフィールドの4種類の値は重複***している。
  - 一つのcsvファイルからはRegionごとにまとめられた4レコードが抽出される。
  - 抽出の際は、***他のフィールドの値は平均値を計算***する。
  - 他のフィールドに設定されている値は、***基本的に実数値***だが、たまに"-"を含んでおり、その値は"0"と読み替える。
  - 最終的に、DataFrameにため込んだデータをcsvファイルとして出力する。

---
## 1. csvファイルのリスト生成

In [1]:
import numpy as np
import glob
import pandas as pd
csv_path_JH = '.\\Data\\03_Sample\\JH_*.csv'
csvs_JH = glob.glob(csv_path_JH)
display(csvs_JH)

['.\\Data\\03_Sample\\JH_H27.csv',
 '.\\Data\\03_Sample\\JH_H28.csv',
 '.\\Data\\03_Sample\\JH_H29.csv',
 '.\\Data\\03_Sample\\JH_H30.csv',
 '.\\Data\\03_Sample\\JH_H31.csv']

---
## 2. csvから必要なデータを取得し、DataFameに格納
  - Dataフォルダ内の

### 2-1. csvを1ファイル読み込んで、必要なデータだけにしたDataFrameを生成する関数を作っておく。

In [2]:
def getDF_CAR(csv):
    #ファイルを読み込み。
    df_org = pd.read_csv(csv, encoding='Shift_JISx0213', low_memory=False)
    
    #1行目には不要なデータが入っているので削除する。
    df = df_org.drop([0,0])
    
    #Yearフィールドの一番上から年度を取得
    year = df.iloc[1]['Year']

    # データは基本的に実数だが、たまに"-"があるので削除。
    # 全体をfloatにしておく。
    df = df.replace('-', 0)
    df = df.astype(float)
    
    # 'Region'フィールドは4種類(1,2,3,4)あり、複数行存在する。
    # 4種類にまとめ、他のフィールドの数値は平均化する。
    # この処理でRegionフィールドがindexになる。
    df = df.groupby('Region').mean()
    
    # 必要なデータのみにする。
    # JH_H31.csvというファイルのデータはヘッダ名が違っているので特殊な処理が必要。
    if csv.find('JH_H31.csv') > 0:
        df = df[['Year','CAR_Jpn', 'CAR_Math']]
        headers = {'CAR_Jpn':'Koku', 'CAR_Math':'Su'}    
    else:
        df = df[['Year','CAR_JpnA', 'CAR_MathA']]
        headers = {'CAR_JpnA':'Koku', 'CAR_MathA':'Su'}

    # ヘッダ名を統一するために変更。
    df = df.rename(columns=headers)
    
    # indexを振りなおし、Regionをフィールドに戻す。
    df = df.reset_index()
    
    # Year、Regionが実数値になっているので、整数に変換。
    df['Year'] = df['Year'].astype(int)
    df['Region'] = df['Region'].astype(int)

    return df   

### 2-2. csvを一つずつ読み込み、必要データをDataFrameに格納する。

In [5]:
# 入れ物を作っておく。
df = pd.DataFrame()

# CSVからデータを読み込み、入れ物に入れる。
for csv in csvs_JH:
    print('processing : '+ csv)
    df_CAR = getDF_CAR(csv)
    df = pd.concat([df, df_CAR], sort=False)

display(df.head(5))
display(df.tail(5))

processing : .\Data\03_Sample\JH_H27.csv
processing : .\Data\03_Sample\JH_H28.csv
processing : .\Data\03_Sample\JH_H29.csv
processing : .\Data\03_Sample\JH_H30.csv
processing : .\Data\03_Sample\JH_H31.csv


,Region,Year,Koku,Su
0,1,2015,44.274370,42.969431
1,2,2015,45.618216,51.053328
2,3,2015,50.199653,47.755236
3,4,2015,45.344529,59.022431
0,1,2016,42.816303,53.701978


,Region,Year,Koku,Su
3,4,2018,38.996858,48.039549
0,1,2019,54.152150,42.745296
1,2,2019,51.654446,58.677429
2,3,2019,49.875922,54.392963
3,4,2019,49.609545,38.084889


---
## 3. DataFrameをcsvに出力する。

In [6]:
df.to_csv('.\\Data\\03_Sample\\JH.csv', index=False)